In [1]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
import json
import pandas as pd
from pymongo import MongoClient

link = 'https://store.steampowered.com/search?category1=998&ndl=1'
head = {'cookie': 'sessionid=5ae342dee8ccf71fc64e6471'}

In [2]:
def get_pagination(game):
    param = {
        'term': game,
        'page': 1,
    }
    total_item = 0
    req = requests.get(link, headers=head, params=param)
    soup = BeautifulSoup(req.text, 'html.parser')
    page_item = soup.find('div', 'search_pagination_right')
    
    if page_item == None: 
        pass
    else:
        page_item = page_item.find_all('a')
        
    
    try:
        total_item = int(page_item[4].text)
    except Exception:
        pass
        try:
            total_item = int(page_item[3].text)
        except Exception:
            pass
            try:
                total_item = int(page_item[2].text)
            except Exception:
                pass
                try:
                    total_item = int(page_item[1].text)
                except Exception:
                    pass
                    try:
                        total_item = int(page_item[0].text)
                    except Exception:
                        pass
    return 1 + total_item

# Game Scraping

In [5]:
def scrap_game_detail():# game_rating_descriptors, game_rating_icons, achievement_block, game_area_metascore
    count = 0
    datas = []

    try:
        for j in range(1, 10000):
            param = {
                'page': j,
            }
            try:
                req = requests.get(link, params=param, headers=head)
                soup = BeautifulSoup(req.text, 'html.parser')
            except:
                pass
            
            conten = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')#tablodaki her bir oyun için
            for i in conten:
                #search page
                #url
                url = i['href']

                #title
                title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

                #price
                try:
                    price = i.find('div', 'col search_price responsive_secondrow').text.strip()
                except Exception:
                    price = 'discount from ' + i.find('span', {'style': 'color: #888888;'}).text.replace(' ', '.') + ' to ' + i.find('div', 'col search_price discounted responsive_secondrow').find('br').next_sibling.strip() + f" ({i.find('div', 'col search_discount responsive_secondrow').text.replace('-', '').strip()})"
                if price == '':
                    price = 'none'

                #release
                release = i.find('div', 'col search_released responsive_secondrow').text
                if release == '':
                    release = 'none'

                #game page
                req_piece = requests.get(url, headers=head)
                soup_piece = BeautifulSoup(req_piece.text, 'html.parser')

                game_id = url.split("/")[4]

                #description
                description = soup_piece.find("div", {"class" : "game_description_snippet"})
                if description == None:
                    description = None 
                else:
                    description = description.text.strip().replace('\n', ' ')

                #header image
                header_image = soup_piece.find("img", {"class" : "game_header_image_full"})
                if header_image == None:
                    header_image = None
                else:
                    header_image = header_image['src']

                #developer
                try:
                    developer = soup_piece.find_all("div", {"class" : "dev_row"})[0]
                    if developer == None:
                        developer = None
                    else:
                        developer = developer.find("a").text
                except:
                    developer = None

                #publisher
                try:
                    publisher = soup_piece.find_all("div", {"class" : "dev_row"})[1]
                    if developer == None:
                        publisher = None
                    else:
                        publisher = publisher.find("a").text
                except:
                    publisher = None

                #tags
                tags = soup_piece.find("div", {"class" : "glance_tags popular_tags"})
                if tags == None:
                    tags = None
                else:
                    tags = [i.text.strip().replace('\n', ' ') for i in tags.find_all("a")][:5]

                #languages
                try:
                    languages = soup_piece.find_all("div", {"id" : "languageTable"})
                    if languages == None:
                        languages = None
                    else:
                        languages = [i.text.strip().replace('\n', ' ') for i in languages[0].find_all("td", {"class":"ellipsis"})]

                except:
                    languages = None

                #background image
                background_image = soup_piece.find("img", {"class" : "game_page_background game"})
                if background_image == None:
                    background_image = None
                else:
                    background_image = cssutils.parseStyle(background_image["style"])['background-image']
                    background_image = background_image.replace('url(', '').replace(')', '')

                sys_req = soup_piece.find("div", {"class" : "game_area_sys_req sysreq_content active"})
                if sys_req == None:
                    sys_req = None
                else:
                    sys_req = sys_req.text.strip().replace('\n', ' ')

                #PEGI Rating
                rating_descriptors = soup_piece.find("div", {"class" : "game_rating_descriptors"})
                if rating_descriptors == None:
                    rating_descriptors = None
                else:
                    rating_descriptors = rating_descriptors.text.split(",")
                    rating_descriptors = [item.strip() for item in rating_descriptors]

                rating_image = soup_piece.find("div", {"class" : "game_rating_icon"})
                if rating_image == None:
                    rating_image = None
                else:
                    rating_image = rating_image.find("img")
                    if rating_image == None:
                         rating_image == None
                    else:
                        rating_image = rating_image["src"]

                #Achievement Count
                achievement_block = soup_piece.find("div", {"id":"achievement_block"})
                if achievement_block == None:
                    achievement_block = None
                else:
                    achievement_block = achievement_block.find("div", {"class":"block_title"})
                    if achievement_block == None:
                        achievement_block = None
                    else:
                        achievement_block = achievement_block.text.strip()

                #Metacritic Score
                metacritic_score = soup_piece.find("div", {"id":"game_area_metascore"})
                if metacritic_score == None:
                    metacritic_score = None
                else:
                    metacritic_score = metacritic_score.find("div", {"class":"score high"})
                    if metacritic_score == None:
                        metacritic_score = None
                    else:
                        metacritic_score = metacritic_score.text.strip()

                #DLC
                all_dlc_list = soup_piece.find("div", {"class":"game_area_dlc_list"})
                if all_dlc_list == None:
                    dlc_info = None
                else:
                    all_dlc_list = all_dlc_list.find_all("a", {"class":"game_area_dlc_row odd ds_collapse_flag ds_collapse_flag_tiny"})
                    dlc_name_list = [item.find("div", {"class":"game_area_dlc_name"}).text.strip() for item in all_dlc_list]
                    dlc_link_list = [item['href'] for item in all_dlc_list]
                    dlc_price_list = [item.find("div", {"class":"game_area_dlc_price"}).text.strip() for item in all_dlc_list]

                    dlc_info = {"dlc_name_list" : dlc_name_list, "dlc_link_list" : dlc_link_list, "dlc_price_list" : dlc_price_list}



                data = {
                    'id':game_id,
                    'title': title,
                    'description': description,
                    'price': price,
                    'release': release,
                    'link': url,
                    'header_image': header_image,
                    'developer': developer,
                    'publisher': publisher,
                    'tags':tags,
                    'languages':languages,
                    'background_image': background_image,
                    'sys_req':sys_req,
                    'rating_descriptors':rating_descriptors,
                    'rating_image':rating_image,
                    'achievement_block':achievement_block,
                    'metacritic_score':metacritic_score,
                    'dlc_info':dlc_info
                }
                datas.append(data)
            print(j)
        return datas
    except:
        return datas

In [6]:
data = scrap_game_detail()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


[{'id': '730',
  'title': 'Counter-Strike: Global Offensive',
  'description': 'Counter-Strike: Global Offensive (CS: GO) expands upon the team-based action gameplay that it pioneered when it was launched 19 years ago. CS: GO features new maps, characters, weapons, and game modes, and delivers updated versions of the classic CS content (de_dust2, etc.).',
  'price': 'Free to Play',
  'release': '21 Aug, 2012',
  'link': 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_230_150_1',
  'header_image': 'https://cdn.cloudflare.steamstatic.com/steam/apps/730/header.jpg?t=1668125812',
  'developer': 'Valve',
  'publisher': 'Valve',
  'tags': ['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action'],
  'languages': ['English',
   'Czech',
   'Danish',
   'Dutch',
   'Finnish',
   'French',
   'German',
   'Hungarian',
   'Italian',
   'Japanese',
   'Korean',
   'Norwegian',
   'Polish',
   'Portuguese - Portugal',
   'Portuguese - Brazil',
   'Romanian',
   'R